In [1]:
# TODO: fix the action encoding

In [2]:
import json
import sys
sys.path.append('../simulated_fqi')
from train_cnfqi import run
import seaborn as sns
import tqdm
import matplotlib.pyplot as plt 
import numpy as np
import configargparse
import torch
import torch.optim as optim
from models.agents import NFQAgent
from models.networks import NFQNetwork, ContrastiveNFQNetwork
# from simulated_fqi import NFQAgent
# from simulated_fqi import NFQNetwork, ContrastiveNFQNetwork
from util import get_logger, load_models, make_reproducible, save_models
import matplotlib.pyplot as plt
import numpy as np
import itertools
import pandas as pd
import pickle
from environments import CartPoleRegulatorEnv

In [3]:
female_test_tuples = pickle.load(open('mimic-iv_tuples/female/trainMgtuples.pkl', 'rb'))
female_train_tuples = pickle.load(open('mimic-iv_tuples/female/testKtuples.pkl', 'rb'))
male_test_tuples = pickle.load(open('mimic-iv_tuples/male/trainKtuples.pkl', 'rb'))
male_train_tuples = pickle.load(open('mimic-iv_tuples/male/testKtuples.pkl', 'rb'))

In [24]:
def generate_rollout(vnum, tuples, group=0):
    trajectory = []
    for i in range(len(tuples['vnum'])):
        vn = tuples['vnum'][i]
        if str(vn) == str(vnum):
            s = tuples['s'][i]
            a = tuples['a'][i]
            a = sum(np.where(a==1))[0]
            ns = tuples['ns'][i]
            r = tuples['r'][i] * 10
            r = int(r)
            
            trajectory.append((s, a, r, ns, False, group))
    
    last_tuple = trajectory[-1]
    new_last_tuple = (last_tuple[0], last_tuple[1], last_tuple[2], last_tuple[3], True)
    trajectory = trajectory[:-1]
    trajectory.append(new_last_tuple)
    return trajectory

def rollout_dataset(ds='train', env='bg'):
    if env == 'bg':
        rollout = []
        for vnum in male_train_tuples['vnum']:
            if ds == 'train':
                traj = generate_rollout(vnum, male_train_tuples, group=0)
            elif ds == 'test':
                traj = generate_rollout(vnum, male_test_tuples, group=0)
            rollout.extend(traj)
        return rollout
    elif env == 'fg':
        rollout = []
        for vnum in female_train_tuples['vnum']:
            if ds == 'train':
                traj = generate_rollout(vnum, female_train_tuples, group=1)
            elif ds == 'test':
                traj = generate_rollout(vnum, female_test_tuples, group=1)
            rollout.extend(traj)
        return rollout


In [26]:
vnum = 20710446
generate_rollout(vnum, female_train_tuples, group=1)[0]


(array([-0.51204251, -0.64756244,  0.53041754,  0.        , -0.48027386,
         2.11335223,  2.6925824 ,  2.94699552, -0.19611614, -0.13522468,
         0.        , -0.13085598,  0.        , -0.30316953,  0.        ,
        -0.0905512 , -1.        ,  1.        , -1.        ,  0.        ,
        -1.        ,  1.        , -0.64190683, -0.03193578,  0.20454902,
        -0.37723692, -2.24971654,  1.45600177,  0.02235021,  0.22518336,
         0.3660076 ,  2.68300862, -0.42257713,  0.28874351, -0.34240223,
         0.09853493, -0.69184817, -0.2187601 ,  0.        , -0.21602341,
        -0.16527101, -0.62065627, -0.07921056, -0.42092031, -0.28261831,
        -0.35120427, -0.19301414, -0.18837312, -0.28675978, -0.36935918,
        -0.19296753, -0.29040202, -0.11001164, -0.07144428, -0.10419916,
        -0.12200976, -0.11622164, -0.13438263,  0.        , -0.14273224,
        -0.2902949 , -0.0474312 ,  0.        ]),
 0,
 -2,
 array([-0.51204251, -0.64756244,  0.53041754,  0.        , -0.480

In [ ]:
is_contrastive=True
epoch = 1
init_experience = 300

# Log to File, Console, TensorBoard, W&B
logger = get_logger()

# Setup agent
nfq_net = ContrastiveNFQNetwork(state_dim=63, is_contrastive=is_contrastive)

if is_contrastive:
    optimizer = optim.Adam(itertools.chain(nfq_net.layers_shared.parameters(), nfq_net.layers_last_shared.parameters()), lr=1e-1)
else:
    optimizer = optim.Adam(nfq_net.parameters(), lr=1e-1)

nfq_agent = NFQAgent(nfq_net, optimizer)

# NFQ Main loop
# A set of transition tuples. Male is background, female is foreground. 
bg_rollouts = rollout_dataset(ds='train', env='bg')
fg_rollouts = rollout_dataset(ds='train', env='fg')

if init_experience > 0:
    bg_rollouts = bg_rollouts[:init_experience]
    fg_rollouts = fg_rollouts[:init_experience]

bg_success_queue = [0] * 3
fg_success_queue = [0] * 3

In [ ]:
bg_rollouts.extend(fg_rollouts)
all_rollouts = bg_rollouts.copy()

In [23]:
state_b, action_b, cost_b, next_state_b, done_b, group_b = zip(*all_rollouts)

ValueError: not enough values to unpack (expected 6, got 5)

In [ ]:
for epoch in range(epoch + 1):

    state_action_b, target_q_values, groups = nfq_agent.generate_pattern_set(all_rollouts)

    loss = nfq_agent.train((state_action_b, target_q_values, groups))

    eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = 0, 0, 0

    if is_contrastive:
        # import ipdb; ipdb.set_trace()
        if nfq_net.freeze_shared:
            eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = nfq_agent.evaluate(
                eval_env_fg, render=False
            )
            for param in nfq_net.layers_fg.parameters():
                assert param.requires_grad == True
            for param in nfq_net.layers_last_fg.parameters():
                assert param.requires_grad == True
            for param in nfq_net.layers_shared.parameters():
                assert param.requires_grad == False
            for param in nfq_net.layers_last_shared.parameters():
                assert param.requires_grad == False
        else:

            for param in nfq_net.layers_fg.parameters():
                assert param.requires_grad == False
            for param in nfq_net.layers_last_fg.parameters():
                assert param.requires_grad == False
            for param in nfq_net.layers_shared.parameters():
                assert param.requires_grad == True
            for param in nfq_net.layers_last_shared.parameters():
                assert param.requires_grad == True
            eval_episode_length_bg, eval_success_bg, eval_episode_cost_bg = nfq_agent.evaluate(
                eval_env_bg, render=False
            )


    else:
        eval_episode_length_bg, eval_success_bg, eval_episode_cost_bg = nfq_agent.evaluate(
            eval_env_bg, render=False
        )
        eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = nfq_agent.evaluate(
            eval_env_fg, render=False
        )

    # bg_success_queue.pop()
    bg_success_queue = bg_success_queue[1:]
    bg_success_queue.append(1 if eval_success_bg else 0)

    fg_success_queue = fg_success_queue[1:]
    fg_success_queue.append(1 if eval_success_fg else 0)

    if sum(bg_success_queue) == 3:
        nfq_net.freeze_shared = True
        if verbose:
            print("FREEZING SHARED")
        if is_contrastive:
            for param in nfq_net.layers_shared.parameters():
                param.requires_grad = False
            for param in nfq_net.layers_last_shared.parameters():
                param.requires_grad = False
            for param in nfq_net.layers_fg.parameters():
                param.requires_grad = True
            for param in nfq_net.layers_last_fg.parameters():
                param.requires_grad = True

            optimizer = optim.Adam(itertools.chain(nfq_net.layers_fg.parameters(), nfq_net.layers_last_fg.parameters()), lr=1e-1)
            nfq_agent._optimizer = optimizer
        # break

    # Print current status
    if verbose:
        logger.info(
            # "Epoch {:4d} | Eval BG {:4d} / {:4f} | Eval FG {:4d} / {:4f} | Train Loss {:.4f}".format(
            #     epoch, eval_env_bg.success_step, eval_episode_cost_bg, eval_env_fg.success_step, eval_episode_cost_fg, loss
            # )
            "Epoch {:4d} | Eval BG {:4d} / {:4f} | Eval FG {:4d} / {:4f} | Train Loss {:.4f}".format(
                epoch, eval_episode_length_bg, eval_episode_cost_bg, eval_episode_length_fg, eval_episode_cost_fg, loss
            )
        )

    if sum(fg_success_queue) == 3:
        if verbose:
            logger.info(
                "Epoch {:4d} | Total Cycles {:6d} | Total Cost {:4.2f}".format(
                    epoch, len(all_rollouts), total_cost
                )
            )
        break

eval_env_bg.step_number = 0
eval_env_fg.step_number = 0

eval_env_bg.max_steps = 1000
eval_env_fg.max_steps = 1000

success = 0
total = 0
for it in range(evaluations):

    # eval_env_bg.save_gif = True
    eval_episode_length_bg, eval_success_bg, eval_episode_cost_bg = nfq_agent.evaluate(eval_env_bg, True)
    # eval_env_bg.create_gif()
    if verbose:
        print(eval_episode_length_bg, eval_success_bg)
    if eval_episode_length_bg == 1000:
        success += 1
    total += 1
    train_env_bg.close()
    eval_env_bg.close()

    # eval_env_fg.save_gif = True
    eval_episode_length_fg, eval_success_fg, eval_episode_cost_fg = nfq_agent.evaluate(eval_env_fg, True)
    # eval_env_fg.create_gif()
    if verbose:
        print(eval_episode_length_fg, eval_success_fg)
    if eval_episode_length_fg == 1000:
        success += 1
    total += 1
    train_env_fg.close()
    eval_env_fg.close()